In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
def process_data(file_name):
    """
    Input: 
        A file path
    Output: 
        words: a list containing all the words in the corpus in lower case. 
    """
    words = [] 
    with open('shakespeare.txt', 'r') as f:
        for line in f:
            words_temp = re.split(r'\W+', line.rstrip().lower())
            for word in words_temp:
                if (len(word) != 0):
                    words.append(word)
     
    return words

In [3]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l)

In [17]:
def get_count(word_l):
    """
    Input:
       A set of words representing the corpus.
    Output:
        A dictionary with kay as a word and value as its frequency
    """
    word_count_dict = {}
    word_count_dict = Counter(word_l)
    return word_count_dict

In [18]:
word_count_dict = Counter(word_l)

In [19]:
def get_probs(word_count_dict):
    """
    Input:
        A dictionary where key is the word and value is its frequency.
    Output:
        A dictionary where keys are the words and the values are the probability that a word will occur. 
    """
    probs = {}  
    m = len(word_l)
    for word, count in word_count_dict.items():
        probs[word] = word_count_dict[word]/m
    return probs

In [20]:
probs = get_probs(word_count_dict)

In [21]:
def delete_letter(word):
    """
    Input:
        A word 
    Output:
        A list of all possible words obtained by deleting 1 character from the input word
    """
    
    delete_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [L + R[1:] for L, R in split_l if len(R) > 0]
    
    return delete_l

In [22]:
def switch_letter(word):
    """
    Input:
        A word 
    Output:
        A list of all possible strings with one adjacent charater switched
    """
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R) > 1]
    
    return switch_l

In [23]:
def replace_letter(word):
    """
    Input:
        A word 
    Output:
        A list of all possible strings where we replaced one letter from the original word
    """
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_set = set([L + l + R[1:] for L, R in split_l if len(R) > 0 for l in letters])
    replace_set.remove(word)
    
    replace_l = sorted(list(replace_set))
    
    return replace_l

In [24]:
def insert_letter(word):
    """
    Input:
        A word 
    Output:
        A set of all possible strings with one new letter inserted at every offset
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [L + l + R for L, R in split_l if len(R) >= 0 for l in letters]
    
    return insert_l

In [25]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        A word
    Output:
        A set of words with one possible edit
    """
    
    edit_one_set = set()
    
    
    inserts = insert_letter(word)
    deletes = delete_letter(word)
    replaces = replace_letter(word)
    options = inserts + deletes + replaces
    for w in options:
        edit_one_set.add(w)
    if allow_switches:
        switches = switch_letter(word)
        for w in switches:
            edit_one_set.add(w)
    
    return edit_one_set

In [26]:
def edit_two_letters(word, allow_switches = True):
    """
    Input:
        A word
    Output:
        A set of strings with all possible two edits
    """
    
    edit_two_set = set()
    
    one_edits = edit_one_letter(word, allow_switches)
    for w in one_edits:
        if (len(w) > 0):
            edits = edit_one_letter(w, allow_switches)
            for w_ in edits:
                edit_two_set.add(w_)
    
    
    return edit_two_set

In [43]:
def get_corrections(word, probs, vocab, n=2):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    if word in vocab:
        n_best.append(word)
        return n_best
    one_edits =  edit_one_letter(word)
    for w in one_edits:
        if w in vocab:
            suggestions.append((w, probs[w]))
    if (len(suggestions) < n):        
        two_edits =  edit_two_letters(word)
        for w in two_edits:
            if w in vocab:
                suggestions.append((w, probs[w]))
    n_best = sorted(suggestions, key = lambda x: -x[1])


    return n_best

In [44]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    m = len(source) 
    n = len(target) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1,m+1): # e
        D[row,0] = D[row-1,0] + 1
        
    for col in range(1,n+1): 
        D[0,col] = D[0, col-1] + 1
        
    for row in range(1,m+1): 
        for col in range(1, n+1):
            r_cost = rep_cost 
            if source[row-1] == target[col-1]:
                r_cost = 0
            D[row,col] = min(D[row-1,col] + del_cost, D[row, col-1] + ins_cost, D[row-1, col-1] + r_cost)
    med = D[m,n]
    
    return D, med

In [45]:
get_corrections("cati", probs, vocab)

[('at', 0.0024433916514343267),
 ('hath', 0.002144962136755325),
 ('can', 0.0019211400007460738),
 ('act', 0.0006528145633603163),
 ('call', 0.0005409034953556906),
 ('hate', 0.00046629611668594026),
 ('eat', 0.00022382213600925132),
 ('care', 0.00022382213600925132),
 ('came', 0.0002051702913418137),
 ('case', 0.0001865184466743761),
 ('cat', 0.00014921475733950087),
 ('camp', 0.00014921475733950087),
 ('late', 0.00014921475733950087),
 ('cat', 0.00014921475733950087),
 ('catch', 0.00013056291267206328),
 ('date', 0.00011191106800462566),
 ('rate', 9.325922333718805e-05),
 ('wait', 9.325922333718805e-05),
 ('cap', 7.460737866975044e-05),
 ('bath', 7.460737866975044e-05),
 ('cut', 7.460737866975044e-05),
 ('gate', 7.460737866975044e-05),
 ('oath', 7.460737866975044e-05),
 ('cast', 7.460737866975044e-05),
 ('acts', 5.595553400231283e-05),
 ('sat', 5.595553400231283e-05),
 ('city', 5.595553400231283e-05),
 ('gait', 3.730368933487522e-05),
 ('pate', 1.865184466743761e-05),
 ('bait', 1.865

In [46]:
get_corrections("mon", probs, vocab)

[('on', 0.0033200283508038947),
 ('man', 0.0013242809713880702),
 ('men', 0.0008766366993695677),
 ('son', 0.0007274219420300668),
 ('won', 0.0001678666020069385),
 ('moan', 9.325922333718805e-05),
 ('moon', 5.595553400231283e-05),
 ('morn', 5.595553400231283e-05),
 ('mov', 3.730368933487522e-05),
 ('con', 1.865184466743761e-05),
 ('non', 1.865184466743761e-05),
 ('mow', 1.865184466743761e-05)]

In [47]:
get_corrections("mat", probs, vocab)

[('at', 0.0024433916514343267),
 ('may', 0.002014399224083262),
 ('man', 0.0013242809713880702),
 ('eat', 0.00022382213600925132),
 ('mad', 0.0001678666020069385),
 ('cat', 0.00014921475733950087),
 ('met', 9.325922333718805e-05),
 ('sat', 5.595553400231283e-05),
 ('mak', 3.730368933487522e-05),
 ('map', 3.730368933487522e-05),
 ('mar', 1.865184466743761e-05),
 ('meat', 1.865184466743761e-05)]